# Introduction
#### The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.
#### The goal of this project is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

#### S&P 500 is made through market capitalisation method which basically gives more weightage to the more larger comapanies then the smaller ones , this project makes an alternate where every company is given similar weightage 

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

### Importing Our List of Stocks
The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [5]:
symbol = "AAPL"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'DQASNA', 'calculationPrice': 'close', 'open': 152, 'openTime': 1662068045526, 'openSource': 'ciliaoff', 'close': 150.36, 'closeTime': 1669164488444, 'closeSource': 'filacifo', 'high': 152.5, 'highTime': 1679811427378, 'highSource': ' e penrmli5dei1aedt cuy', 'low': 150.36, 'lowTime': 1676838625567, 'lowSource': 'eri5  1edildna teeypcmu', 'latestPrice': 151.67, 'latestSource': 'Close', 'latestTime': 'August 13, 2021', 'latestUpdate': 1659543710585, 'latestVolume': 60360775, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 154.1, 'delayedPriceTime': 1694104841689, 'oddLotDelayedPrice': 153.04, 'oddLotDelayedPriceTime': 1655174926974, 'extendedPrice': 153.34, 'extendedChange': 0.04, 'extendedChangePercent': 0.00028, 'extendedPriceTime': 1693181214573, 'previousClose': 149.51, 'previousVolume': 49016215, 'change': 3.05, 'changePercent': 0.02141, 'volume': 61461597, 'iexMarketPercent

In [6]:
price = data["latestPrice"]
market_cap = data["marketCap"]


In [10]:
my_columns = ['Ticker' , 'Stock Price' , 'Market Capitalization' , 'Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)


In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy


In [13]:
for stock in stocks["Ticker"]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(pd.Series(
    [
        stock,
        data["latestPrice"],
        data["marketCap"],
        "n/a"
        
    ],index  = my_columns), ignore_index=True)
    

In [15]:
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,162.64,50183596847,n/a
1,AAL,20.45,13855481272,n/a
2,AAP,213.00,14279747455,n/a
3,AAPL,150.98,2557974126551,n/a
4,ABBV,116.96,205830634343,n/a


## Using batching as it is less time consuming and usually cheaper too

In [16]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [23]:
symbol_group = list(chunks(stocks["Ticker"],100))
symbol_strings =[]
for i in range(0,len(symbol_group)):
    symbol_strings.append(",".join(symbol_group[i]))
    
final_dataframe = pd.DataFrame(columns=my_columns)   

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(pd.Series(
        [
            symbol,
            data[symbol]["quote"]['latestPrice'],
            data[symbol]["quote"]["marketCap"],
            "n/a"
            
        ],index=my_columns),ignore_index=True)
    
    
    
    

In [24]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,163.94,48130012370,n/a
1,AAL,20.71,13505132782,n/a
2,AAP,214.90,14202852492,n/a
3,AAPL,155.41,2581216912531,n/a
4,ABBV,117.38,212552813829,n/a
...,...,...,...,...
500,YUM,139.77,40186503667,n/a
501,ZBH,149.53,31302845392,n/a
502,ZBRA,592.40,30827887295,n/a
503,ZION,56.13,9104014352,n/a


In [34]:
portfolio_size = input("please enter the size of your portfolio :")

try:
    val = float(portfolio_size)
except ValueError:
    print("that is not a number \nplease enter a number")
    portfolio_size = input("please enter the size of your portfolio : ")
    val = float(portfolio_size)
    

please enter the size of your portfolio :10000000


In [35]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i , "Number of shares to buy"] = math.floor(position_size / final_dataframe.loc[i , "Stock Price"])
final_dataframe



,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,163.94,48130012370,120
1,AAL,20.71,13505132782,956
2,AAP,214.90,14202852492,92
3,AAPL,155.41,2581216912531,127
4,ABBV,117.38,212552813829,168
...,...,...,...,...
500,YUM,139.77,40186503667,141
501,ZBH,149.53,31302845392,132
502,ZBRA,592.40,30827887295,33
503,ZION,56.13,9104014352,352


## Formatting in excel

In [38]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)


In [40]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    "font_color":font_color,
    "bg_color":background_color,
    "border":1
})

dollar_format = writer.book.add_format(
{
    "num_format":"$0.00",
    "font_color":font_color,
    "bg_color":background_color,
    "border":1
})

integer_format = writer.book.add_format(
{
    "num_format":'0',
    "font_color":font_color,
    "bg_color":background_color,
    "border":1
})



We can use the set_column method applied to the writer.sheets['Recommended Trades'] object to apply formats to specific columns of our spreadsheets.

Here's an example:

writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B

                     18, #This tells the method to apply a column width of 18 pixels
                     
                     string_format #This applies the format 'string_format' to the column
                    )

In [42]:
column_formats = {
    "A" : ["Ticker" , string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

In [45]:
for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
    

In [46]:
writer.save()